# Connect to Indy by IndyDCP

* robot_ip: IP address of robot (actually, STEP in IndyCB)
* name: string represents robot model
    * Indy7: "NRMK-Indy7"
    * IndyRP2: "NRMK-IndyRP2"
    


In [1]:
from indy_utils import indydcp_client as client

import json
from time import sleep
import threading
import numpy as np

robot_ip = "192.168.0.3"  # Robot (Indy) IP
robot_name = "NRMK-Indy7"  # Robot name (Indy7)
# robot_name = "NRMK-IndyRP2"  # Robot name (IndyRP2)

# Create class object
indy = client.IndyDCPClient(robot_ip, robot_name)

In [2]:
# Connect to robot
indy.connect()

# Disconnect with robot
# recommanded for finishing program properly
indy.disconnect()

Connect: Server IP (141.223.199.180)


# IndyDCP command function examples

# Get robot status

## Current Indy status
* Get robot status each

In [11]:
indy.connect()

status = indy.get_robot_status()
print(status)

indy.disconnect()

Connect: Server IP (141.223.199.180)
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}


In [12]:
status['resetting']

0

## Emergency stop and reset robot
* **emergency_stop()** : enter the emergency mode
* **reset_robot()**: reset from the emergency mode

In [13]:
indy.connect()
indy.stop_emergency()

status = indy.get_robot_status()
print("Emergency stop")
print("is in resetting? ", status['resetting'])
print("is robot ready? ", status['ready'])


indy.reset_robot()
status = indy.get_robot_status()
print("Resetting robot")
print("is in resetting? ", status['resetting'])
print("is robot ready? ", status['ready'])


sleep(5)
status = indy.get_robot_status()
print("Reset robot done")
print("is in resetting? ", status['resetting'])
print("is robot ready? ", status['ready'])

indy.disconnect()

Connect: Server IP (141.223.199.180)
Emergency stop
is in resetting?  0
is robot ready?  0
Resetting robot
is in resetting?  1
is robot ready?  0
Reset robot done
is in resetting?  0
is robot ready?  1


## Joint servo and brake control
* **get_servo_state()**: get robot's joint servo state
* **set_servo()**: set robot's joint servo state

In [8]:
indy.connect()
servo, brake = indy.get_servo_state()
print("Servo: ", servo)
print("Brake: ", brake)

print("Turn off joint 6th servo")
indy.set_servo([1, 1, 1, 1, 1, 0])
sleep(0.5)
print("Turn on joint 6th brake")
indy.set_brake([0, 0, 0, 0, 0, 1])

servo, brake = indy.get_servo_state()
print("Servo: ", servo)
print("Brake: ", brake)
indy.disconnect()

Connect: Server IP (192.168.3.134)
Servo:  [1, 1, 1, 1, 1, 1]
Brake:  [0, 0, 0, 0, 0, 0]
Turn off joint 6th servo
Turn on joint 6th brake
Header check fail (cmdId): Request 4, Response 9999
ErrorCode 21: Not proper robot state
Servo:  [1, 1, 1, 1, 1, 0]
Brake:  [0, 0, 0, 0, 0, 1]


## Direct teaching mode
* **change_to_direct_teaching()** : turn on direct teaching mode
* **finish_direct_teaching()** : turn off direct teaching mode

In [9]:
indy.connect()

print("Start direct teaching mode")
indy.direct_teaching(True)

indy.disconnect()

Connect: Server IP (141.223.199.180)
Start direct teaching mode


In [10]:
indy.connect()

indy.direct_teaching(False)
print("Finish direct teaching mode")

indy.disconnect()

Connect: Server IP (141.223.199.180)
Finish direct teaching mode


## Get Indy data and properties
* **get_robot_running_time()**: time after robot turns on by sec
* **get_cmode()** : get control mode of robot
    * cmode=0: stationary
    * cmode=1: joint move
    * cmode=2: task move
    * cmode=3: direct teaching
    * ... (refer to http://docs.neuromeka.com/2.3.0/en/IndyAPI/section2/#robotcontrolshareddata for details)

In [10]:
indy.connect()
print(indy.get_robot_running_time(), "sec")
print(indy.get_cmode())
indy.disconnect()

Connect: Server IP (192.168.3.134)
13128.3525 sec
0


## Robot properties
* **set_collision_level()**: set collision detection sensitivity level
* **set_joint_speed_level()**: set joint move velocity level 
* **set_task_speed_level()**: set task move velocity level
* **set_joint_blend_radius()**: set joint move blending radius (3~27 [deg])
* **set_task_blend_radius()**: set task move blending radius (0.02~0.2 [meter])  

In [11]:
indy.connect()
indy.set_collision_level(5)
indy.set_joint_vel_level(3)
indy.set_task_vel_level(3)
indy.set_joint_blend_radius(20)
indy.set_task_blend_radius(0.2)

print(indy.get_collision_level())
print(indy.get_joint_vel_level())
print(indy.get_task_vel_level())
print(indy.get_joint_blend_radius())
print(indy.get_task_blend_radius())
indy.disconnect()

Connect: Server IP (141.223.199.180)
5
3
3
20.0
0.2


* **get_joint_pos()**: angles of each joint [deg]
* **get_task_pos()**: task space position and orientation [meter, deg]

In [12]:
indy.connect()
j_pos = indy.get_joint_pos()
t_pos = indy.get_task_pos()

print(j_pos)
print(t_pos)
indy.disconnect()

Connect: Server IP (192.168.3.134)
[89.99682010131491, 4.89995714239108e-14, -89.9988200750943, -3.2772339430097204e-15, -90.00182003576363, 2.2220876110610958e-14]
[0.18651942446706263, 0.34998710146675666, 0.5220072077653144, 1.7611327592275283e-14, -179.9993598891421, 89.99682010131488]


* **get_joint_vel_level()**: angular velocity of each joint [deg/s]
* **get_task_vel_level()**: task space velocity [m/s, deg/s]
* **get_control_torque()**: joint torque applied by the controller [Nm]

In [12]:
indy.connect()
j_vel = indy.get_joint_vel_level()
t_vel = indy.get_task_vel_level()
torque = indy.get_control_torque()

print(j_vel)
print(t_vel)
print(torque)
indy.disconnect()

Connect: Server IP (141.223.199.180)
3
3
[-1.0928578116571668e-19, 0.004674716095771055, 0.004203468635986184, -8.097916054948144e-16, 0.0014535208197491176, -4.284995417780908e-19]


### Error information
* **get_last_emergency_info()**: get last log message with error code and specific error value

In [14]:
indy.connect()
e_code, e1, e2 = indy.get_last_emergency_info()
print(e_code)
print(e1)
print(e2)
indy.disconnect()

Connect: Server IP (192.168.3.134)
-1
[0, 0, 0]
[0.0, 0.0, 0.0]


## Motion commands
### Warning! Indy will move. Please clean the area around the robot
* **go_home()**: move to home position
* **go_zero()**: move to zero position
* **stop_motion()**: stop moving
* **execute_move(cmd_name)**: execute specific move command saved in Conty's 1st default program

In [15]:
indy.connect()

Connect: Server IP (192.168.3.134)


True

In [16]:
indy.go_home()

In [17]:
indy.go_zero()

In [18]:
indy.stop_motion()

In [20]:
cmd_name = "j_move1" # cmd_name is a move name in Conty program tree of 1st default program 
indy.execute_move(cmd_name)

In [21]:
indy.disconnect()

### Joint move and Task move
* **joint_move_to()**: joint move to defined absolute position [deg]
* **joint_move_by()**: joint move to defined position by relative value based on current position [deg]
* **task_move_to()**: task move to defined absolute position [meter, deg]
* **task_move_by()**: task move to defined position by relative value based on current position [meter, deg]

In [22]:
indy.connect()

Connect: Server IP (192.168.3.134)


True

In [23]:
j_pos1 = indy.get_joint_pos()
t_pos1 = indy.get_task_pos()
print("j_pos1", j_pos1)
print("t_pos1", t_pos1)

j_pos1 [92.59613439830707, -1.8434750478895254, -108.86081578922634, 3.0990637729111455e-05, -69.29679914569108, 4.44921440047707]
t_pos1 [0.1708233584431076, 0.34996463235861464, 0.3980262907463989, 5.565404859292633e-05, -179.99891105301998, 88.14690904099503]


In [24]:
j_pos1[5] += 20  # move for Indy7
# j_pos1[6] += 20  # move for IndyRP2
indy.joint_move_to(j_pos1)  # Move 6th joint

In [25]:
j_pos_rel = [0, 0, 0, 0, 0, -20]  # move for Indy7
# j_pos_rel = [0, 0, 0, 0, 0, 0, -20]  # move for IndyRP2
indy.joint_move_by(j_pos_rel)

In [26]:
t_pos1[2] += 0.1
indy.task_move_to(t_pos1)  # Move along z-axis

In [27]:
t_pos_rel = [0, 0, -0.1, 0, 0, 0]
indy.task_move_by(t_pos_rel)

In [28]:
indy.disconnect()

## Default program and current program control

In [29]:
indy.connect()

Connect: Server IP (192.168.3.134)


True

In [30]:
indy.set_default_program(1)

print("Current default program index: ", indy.get_default_program_idx())

print("Start the default program")
indy.start_default_program()

sleep(1)
print("Pause current program")
indy.pause_current_program()

sleep(1)
print("Resume current program")
indy.resume_current_program()

sleep(1)
print("Stop current program")
indy.stop_current_program()

Current default program index:  1
Start the default program
Pause current program
Resume current program
Stop current program


In [31]:
indy.disconnect()

## Interfacing with external devices using DIO and AIO

## Analog input and output
* Analog input/output 0-10V: 0-10000

In [2]:
indy.connect()

Connect: Server IP (141.223.199.180)


True

In [3]:
print(indy.get_ai(0))   # Get analog input from AI channel 1
print(indy.get_ai(1))   # Get analog input from AI channel 2

0
0


In [3]:
indy.set_ao(idx=0, val=10000) # 10V output to AI channel 1
indy.set_ao(idx=1, val=2000)  # 2V output to AI channel 2

In [3]:
indy.disconnect()

## Digital input

In [7]:
indy.connect()

dis = indy.get_di()
print("[DI0, DI20]: [{}, {}]".format(dis[0], dis[20]))

indy.disconnect()

Connect: Server IP (141.223.199.180)
[DI0, DI20]: [0, 1]


## Digital output

In [8]:
indy.connect()

Connect: Server IP (141.223.199.180)


True

In [9]:
idx = 8

indy.set_do(8, True)
sleep(1)
indy.set_do(8, False)

In [37]:
dos_all = [False]*32   # 32 boolean list

dos_all[2] = True
dos_all[4] = True
dos_all[8] = True
dos_all[12] = True

indy.set_smart_dos(dos_all)
sleep(2)


dos_all = [False]*32
indy.set_smart_dos(dos_all)

In [10]:
indy.disconnect()

## EndTool output

In [7]:
# endtool_type:
# 0: NPN, 1: PNP, 2: Not use, 3: eModi

endtool_type = 0
indy.set_endtool_do(endtool_type, 1)  # val: 0(off), 1(on)
sleep(1)
indy.get_endtool_do(endtool_type)

1

In [18]:
indy.disconnect()

## Direct variables
### Assign/Read direct variable
* Please refer to http://docs.neuromeka.com/2.3.0/en/IndyAPI/section3/ for details about direct variables

In [12]:
indy.connect()

Connect: Server IP (192.168.3.126)


True

In [38]:
DIRECT_VAR_TYPE_BYTE       = 0   # 1 Byte unsigned integer (0-255)
DIRECT_VAR_TYPE_WORD       = 1   # 2 Byte integer (-32768 - 32767)
DIRECT_VAR_TYPE_DWORD      = 2   # 4 Byte integer (-2,147,483,648 - 2,147,483,647)
DIRECT_VAR_TYPE_LWORD      = 3   # 8 Byte integer (-9223372036854775808 to 9223372032559808511)
DIRECT_VAR_TYPE_FLOAT      = 4   # 4 Byte floating number
DIRECT_VAR_TYPE_DFLOAT     = 5   # 8 Byte floating number
DIRECT_VAR_TYPE_MODBUS_REG = 10  # ModbusTCP, 2 Byte unsigned integer (0 - 65535)


# B
indy.write_direct_variable(dv_type=DIRECT_VAR_TYPE_BYTE, dv_addr=231, val=12)
val = indy.read_direct_variable(dv_type=DIRECT_VAR_TYPE_BYTE, dv_addr=231)
print("B231=", val)

# W
indy.write_direct_variable(dv_type=DIRECT_VAR_TYPE_WORD, dv_addr=112, val=-4322)
val = indy.read_direct_variable(dv_type=DIRECT_VAR_TYPE_WORD, dv_addr=112)
print("W112=", val)

# I
indy.write_direct_variable(dv_type=DIRECT_VAR_TYPE_DWORD, dv_addr=231, val=-223)
res = indy.read_direct_variable(dv_type=DIRECT_VAR_TYPE_DWORD, dv_addr=231)
print("I231=", res)

# L
indy.write_direct_variable(dv_type=DIRECT_VAR_TYPE_LWORD, dv_addr=58, val=9223372032559812)
val = indy.read_direct_variable(dv_type=DIRECT_VAR_TYPE_LWORD, dv_addr=58)
print("L058=", val)

# F
indy.write_direct_variable(dv_type=DIRECT_VAR_TYPE_FLOAT, dv_addr=998, val=-847.213)
val = indy.read_direct_variable(dv_type=DIRECT_VAR_TYPE_FLOAT, dv_addr=998)
print("F998=", val)

# D
indy.write_direct_variable(dv_type=DIRECT_VAR_TYPE_DFLOAT, dv_addr=700, val=327697.113)
val = indy.read_direct_variable(dv_type=DIRECT_VAR_TYPE_DFLOAT, dv_addr=700)
print("D700=", val)

# M (Modbus)
indy.write_direct_variable(dv_type=DIRECT_VAR_TYPE_MODBUS_REG, dv_addr=120, val=1000)
val = indy.read_direct_variable(dv_type=DIRECT_VAR_TYPE_MODBUS_REG, dv_addr=120)
print("M700=", val)

[  0   0   0   0 231   0   0   0  12   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
9
B231= 12
[  1   0   0   0 112   0   0   0  30 239   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0  

In [39]:
indy.write_direct_variables(dv_type=DIRECT_VAR_TYPE_BYTE, dv_addr=0, dv_len=10, val=[1, 2, -3, 4, 5, -6, 7, 8, -9, 10])
val = indy.read_direct_variables(dv_type=DIRECT_VAR_TYPE_BYTE, dv_addr=0, dv_len=10)
print("B0-9=", val)

indy.write_direct_variables(dv_type=DIRECT_VAR_TYPE_WORD, dv_addr=0, dv_len=10, val=[10, 20, -30, 40, 50, -60, 70, 80, -90, 100])
val = indy.read_direct_variables(dv_type=DIRECT_VAR_TYPE_WORD, dv_addr=0, dv_len=10)
print("W0-9=", val)

indy.write_direct_variables(dv_type=DIRECT_VAR_TYPE_DWORD, dv_addr=0, dv_len=10, val=[100, 200, -300, 400, 500, -600, 700, 800, -900, 1000])
val = indy.read_direct_variables(dv_type=DIRECT_VAR_TYPE_DWORD, dv_addr=0, dv_len=10)
print("I0-9=", val)

indy.write_direct_variables(dv_type=DIRECT_VAR_TYPE_LWORD, dv_addr=0, dv_len=10, val=[1000, 2000, -3000, 4000, 5000, -6000, 7000, 8000, -9000, 10000])
val = indy.read_direct_variables(dv_type=DIRECT_VAR_TYPE_LWORD, dv_addr=0, dv_len=10)
print("L0-9=", val)

indy.write_direct_variables(dv_type=DIRECT_VAR_TYPE_FLOAT, dv_addr=0, dv_len=10, val=[1.1, 2.2, -3.3, 4.4, 5.5, -6.6, 7.7, 8.8, -9.9, 10.1])
val = indy.read_direct_variables(dv_type=DIRECT_VAR_TYPE_FLOAT, dv_addr=0, dv_len=10)
print("F0-9=", val)

indy.write_direct_variables(dv_type=DIRECT_VAR_TYPE_DFLOAT, dv_addr=0, dv_len=10, val=[0.01, 0.02, -0.03, 0.04, 0.05, -0.06, 0.07, 0.08, -0.09, 0.1])
val = indy.read_direct_variables(dv_type=DIRECT_VAR_TYPE_DFLOAT, dv_addr=0, dv_len=10)
print("D0-9=", val)

indy.write_direct_variables(dv_type=DIRECT_VAR_TYPE_MODBUS_REG, dv_addr=0, dv_len=10, val=[11, 22, -33, 44, 55, -66, 77, 88, -99, 111])
val = indy.read_direct_variables(dv_type=DIRECT_VAR_TYPE_MODBUS_REG, dv_addr=0, dv_len=10)
print("M0-9=", val)

B0-9= [1, 2, 253, 4, 5, 250, 7, 8, 247, 10]
W0-9= [10, 20, -30, 40, 50, -60, 70, 80, -90, 100]
I0-9= [100, 200, -300, 400, 500, -600, 700, 800, -900, 1000]
L0-9= [1000, 2000, -3000, 4000, 5000, -6000, 7000, 8000, -9000, 10000]
F0-9= [1.1, 2.2, -3.3, 4.4, 5.5, -6.6, 7.7, 8.8, -9.9, 10.1]
D0-9= [0.01, 0.02, -0.03, 0.04, 0.05, -0.06, 0.07, 0.08, -0.09, 0.1]
M0-9= [11, 22, 65503, 44, 55, 65470, 77, 88, 65437, 111]


In [40]:
indy.disconnect()

## Extended IndyDCP

## External Trajectory Move
### Data ordering (Joint)
* qd[0] > qd[1] > qd[2] > qd[3] > qd[4] > qd[5] > qdotd[0] > qdotd[1] > ... > qdotd[5] > qddotd[0] > qddotd[1] > ... > next time

### Text type trajectory move 

In [41]:
indy.connect()

Connect: Server IP (192.168.3.134)


True

In [42]:
# Load predefined trajectory data to list
file_path = 'traj_data/indy7_joint_small.txt'  
dat = np.loadtxt(file_path)
dat_flt = dat.flatten()
traj_data = dat_flt.tolist()

# Options
traj_type = 1  # 1 for joint move, 2 for task move
traj_freq = 4000  # control rate
dat_size = 6  # DOF


# External trajectory move with text data
indy.move_ext_traj_txt(traj_type, traj_freq, dat_size, traj_data)

In [43]:
indy.disconnect()

### Binary type trajectory move

In [44]:
indy.connect()

Connect: Server IP (192.168.3.134)


True

In [45]:
# Load predefined trajectory data to list
file_path = 'traj_data/indy7_joint_small.txt'
dat = np.loadtxt(file_path)
dat_flt = dat.flatten()
traj_data = dat_flt.tolist()

# Options
traj_type = 1  # 1 for joint move, 2 for task move
traj_freq = 4000  # control rate
dat_size = 6  # DOF


# External trajectory move with binary data
indy.move_ext_traj_bin(traj_type, traj_freq, dat_size, traj_data)

In [10]:
indy.disconnect()

### Save trajectory text file with options

In [45]:
# Load predefined trajectory data to list
file_path = 'traj_data/indy7_joint_small.txt'
dat = np.loadtxt(file_path)
dat_flt = dat.flatten()
traj_data = dat_flt.tolist()

# Options
traj_type = 1  # 1 for joint move, 2 for task move
traj_freq = 4000  # control rate
dat_size = 6  # DOF

# Default options (need not to be modified)
opt_len = 5
dat_num = 3 # (qd, qdotd, qddotd) or (pd, pdotd, pddotd)
dat_len = len(traj_data)

# Save options with trajectory data
ext_data_size = opt_len + dat_len
ext_data = [None] * ext_data_size
ext_data[0] = traj_type
ext_data[1] = traj_freq
ext_data[2] = dat_num
ext_data[3] = dat_size
ext_data[4] = int(dat_len/(dat_size*dat_num))  # traj_len
ext_data[5:-1] = traj_data

ext_data_str = ' '.join(str(e) for e in ext_data)


# Make binary file
newFile = open("traj_data/indy7_joint_small_ext.txt", "wt") # It should be moved to STEP!!

# Write to file
newFile.write(ext_data_str)

# After copy file to STEP, please this command via SSH (e.g. putty)
# chmod 777 * /home/user/Downloads/

585077

### Save trajectory text file to binary file

In [40]:
# Load predefined trajectory data to list
file_path = 'traj_data/indy7_joint_small.txt'
dat = np.loadtxt(file_path)
dat_flt = dat.flatten()
traj_data = dat_flt.tolist()

# Options
traj_type = 1  # 1 for joint move, 2 for task move
traj_freq = 4000  # control rate
dat_size = 6  # DOF

# Default options (need not to be modified)
opt_len = 5
dat_num = 3 # (qd, qdotd, qddotd) or (pd, pdotd, pddotd)
dat_len = len(traj_data)

# Save options to data array data
opt_data = [None] * opt_len
opt_data[0] = traj_type
opt_data[1] = traj_freq
opt_data[2] = dat_num
opt_data[3] = dat_size
opt_data[4] = int(dat_len/(dat_size*dat_num))  # traj_len

ext_data1 = np.array(opt_data).tobytes()
ext_data2 = np.array(traj_data).tobytes()
req_ext_data = ext_data1 + ext_data2


# Make binary file
newFile = open("traj_data/indy7_joint_small", "wb") # It should be moved to STEP!!

# Write to file
newFile.write(req_ext_data)

# After copy file to STEP, please this command via SSH (e.g. putty)
# chmod 777 * /home/user/Downloads/

409124

### Trajectory move by reading binary file in STEP

In [48]:
file_path = '/home/user/Downloads/indy7_joint_small'

indy.connect()
indy.move_ext_traj_bin_file(file_path)

Connect: Server IP (192.168.3.134)


In [ ]:
indy.disconnect()

### Trajectory move by reading text file in STEP

In [49]:
file_path = '/home/user/Downloads/indy7_joint_small_ext.txt'

indy.connect()
indy.move_ext_traj_txt_file(file_path)

Connect: Server IP (192.168.3.134)


In [50]:
indy.disconnect()